<a href="https://colab.research.google.com/github/VanJoaoPedro/Deep-Learning-Python/blob/master/Emotion_detector/video_test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries


In [ ]:
import cv2
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import zipfile

# Accessing the files

In [ ]:
path = '/content/drive/My Drive/teste_modelo.zip'
zip_object = zipfile.ZipFile(file=path)
zip_object.extractall('./')
zip_object.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/drive/My Drive/teste_modelo.zip' mode='r'>>

# Loading the model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/My Drive/modelo_04_expressoes.h5')

In [ ]:
model.summary()

# Loading video

In [ ]:
file_video = '/content/video_teste05.mp4'
capture = cv2.VideoCapture(file_video)

connected, video = capture.read()
print(connected, video.shape)

True (480, 854, 3)


# Video resizing to decrease processing time

In [ ]:
resizing = True
maximum_width = 600

if (resizing and video.shape[1] > maximum_width):
    proportion = video.shape[1] / video.shape[0]
    video_width = maximum_width
    video_height = int(video_width / proportion)
else:
    video_width = video.shape[1]
    video_height = video.shape[0]

# Defining the video settings

**Codecs :**

A FourCC is a four-byte string used to uniquely identify data formats

Most used codecs: XVID, MP4V, MJPG, DIVX, X264 ...

Example: To save in mp4 format, the MP4V codec is used (The file name also needs the .mp4 extension)

In [ ]:
file_name = 'result_video_test.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 64
output_video = cv2.VideoWriter(file_name, fourcc, fps, (video_width, video_height))

# Video processing and recording of the result

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
haarcascade_faces = '/content/haarcascade_frontalface_default.xml'
small_font, medium_font = 0.4, 0.7
font = cv2.FONT_HERSHEY_SIMPLEX
expression = ['Raiva', 'Nojo', 'Medo', 'Feliz', 'Triste', 'Surpreso', 'Neutro']

while (cv2.waitKey(1) <0 ):
    connected, frame = capture.read()

    if not connected:
        break
    
    t = time.time()

    if resizing:
        frame = cv2.resize(frame, (video_width, video_height))

    face_cascade = cv2.CascadeClassifier(haarcascade_faces)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))

    if len(faces) > 0:
        for (x, y, w, h) in faces:
            frame = cv2.rectangle(frame, (x,y), (x + w, y + h + 10), (255, 50, 50), 2)
            region_of_interest = gray[y:y + h, x:x + w]
            region_of_interest = cv2.resize(region_of_interest, (48, 48))
            region_of_interest = img_to_array(region_of_interest)
            region_of_interest = np.expand_dims(region_of_interest, axis=0)

            result = model.predict(region_of_interest)[0]
            print(result)

            if result is not None:
                final_result = np.argmax(result)
                cv2.putText(frame, expression[final_result], (x, y - 10), font, medium_font, (255, 255, 255), 1, cv2.LINE_AA)
    
    cv2.putText(frame, "frame processado em {:.2f} segundos ".format(time.time() - t), (20, video_height - 20), font, small_font,(250, 250, 250), 0, lineType=cv2.LINE_AA)
    cv2_imshow(frame)
    output_video.write(frame)
output_video.release()
cv2.destroyAllWindows()

In [ ]:
print(time.time())